In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#Problem Statement : How many people are more at risk for covid based on their location(Based on NYC boroughs)

In [2]:
#load the dataset
covid_data = pd.read_csv('/content/COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv')

In [3]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915 entries, 0 to 1914
Data columns (total 55 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   date_of_interest                 1915 non-null   object
 1   CASE_COUNT                       1915 non-null   int64 
 2   PROBABLE_CASE_COUNT              1915 non-null   int64 
 3   HOSPITALIZED_COUNT               1915 non-null   int64 
 4   DEATH_COUNT                      1915 non-null   int64 
 5   CASE_COUNT_7DAY_AVG              1915 non-null   int64 
 6   ALL_CASE_COUNT_7DAY_AVG          1915 non-null   int64 
 7   HOSP_COUNT_7DAY_AVG              1915 non-null   int64 
 8   DEATH_COUNT_7DAY_AVG             1915 non-null   int64 
 9   BX_CASE_COUNT                    1915 non-null   int64 
 10  BX_PROBABLE_CASE_COUNT           1915 non-null   int64 
 11  BX_HOSPITALIZED_COUNT            1915 non-null   int64 
 12  BX_DEATH_COUNT                   1

In [4]:
#Loads the dataset
covid_data.describe()
covid_data.head()

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,...,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,INCOMPLETE
0,02/29/2020,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,03/01/2020,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,03/02/2020,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,03/03/2020,1,0,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,03/04/2020,5,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#feature engineering
# Assign covid_data to covid_data_long before performing feature engineering
# covid_data = covid_data_long.copy() # Uncomment this line if you intended to use covid_data_long

# Let's work with a fresh copy of the data loaded from the CSV to ensure 'borough' is present
# Assuming covid_data from cell 2 is the desired starting point
# Make sure the covid_data variable holds the DataFrame loaded from the CSV
# If there were previous cells that modified covid_data and removed 'borough',
# re-run cell 2 or ensure you are starting with the correct DataFrame.
# For this fix, we assume covid_data already holds the correct data.

# It's generally a good practice to create a copy to avoid modifying the original DataFrame
covid_data_processed = covid_data.copy()

# Convert date column
covid_data_processed['date_of_interest'] = pd.to_datetime(covid_data_processed['date_of_interest'])

# Extract month and weekday
covid_data_processed['month'] = covid_data_processed['date_of_interest'].dt.month #extracts the month from the date column
covid_data_processed['weekday'] = covid_data_processed['date_of_interest'].dt.dayofweek #extracts the day of the week

# Check if 'borough' column exists before applying get_dummies
if 'borough' in covid_data_processed.columns:
    # converts borough column into numeric column
    covid_data_processed = pd.get_dummies(covid_data_processed, columns=['borough'], descending = True, drop_first=True)
else:
    print("Error: 'borough' column not found in the DataFrame.")
    # Handle this case - perhaps load the correct data or skip get_dummies

# Display head of the processed DataFrame
print(covid_data_processed.head())

# Now use covid_data_processed for the train-test split
# Reassign covid_data to the processed DataFrame if you want to use that name in subsequent cells
# Or, update the train-test split cell to use covid_data_processed
covid_data = covid_data_processed

Error: 'borough' column not found in the DataFrame.
  date_of_interest  CASE_COUNT  PROBABLE_CASE_COUNT  HOSPITALIZED_COUNT  \
0       2020-02-29           1                    0                   1   
1       2020-03-01           0                    0                   1   
2       2020-03-02           0                    0                   2   
3       2020-03-03           1                    0                   8   
4       2020-03-04           5                    0                   2   

   DEATH_COUNT  CASE_COUNT_7DAY_AVG  ALL_CASE_COUNT_7DAY_AVG  \
0            0                    0                        0   
1            0                    0                        0   
2            0                    0                        0   
3            0                    0                        0   
4            0                    0                        0   

   HOSP_COUNT_7DAY_AVG  DEATH_COUNT_7DAY_AVG  BX_CASE_COUNT  ...  \
0                    0                     0

In [6]:
#Train- Test split
#We split our data into 80% for training and 20% for testing. This helps evaluate how well the model performs on unseen data.
#X = features (independent variables)
# Y = target (what you want to predict)

#test_size : 20% goes into testing, 80% into training
#random_state : ensures you get the same split every time
# X_train, y_train : used to fit the model
# X_test, Y_test : used to evaluate model
# Assuming 'covid_data_processed' has been created and contains the dataframe
# with feature engineering applied (from the previous cell).
features = covid_data_processed.drop(['date_of_interest','death_count'], axis = 1)
target = covid_data_processed['death_count']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

KeyError: "['death_count'] not found in axis"

In [ ]:
y_pred = model.predict(X_test) #Use the model to predict on test data

#evaluates how well it did
#Average squared error
#R^2 score
mse = mean_squared_error(y_test, y_pred) #mean squared error tells you how wrong your model is
r2 = r2_score(y_test, y_pred)# tells you how useful your model is

print("Mean Squared Error:", mse)
print("R^2 Score", r2)


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)